In [1]:
from datetime import datetime

import numpy as np
import pandas as pd

from scipy.integrate import odeint
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error

import plotly.express as px
import plotly.graph_objects as go

In [2]:
# MY TRY
URL = "https://raw.githubusercontent.com/owid/monkeypox/main/owid-monkeypox-data.csv"
df = pd.read_csv(URL, parse_dates=["date"])[["location", "date", "total_cases", "total_deaths"]].drop_duplicates().fillna(0)


df = df[df["location"]=="World"].drop(columns=["location"]).set_index("date")
df = df.rename(columns={"total_cases": "cases",
                  "total_deaths": "deaths"})
df.resample("d").interpolate("linear")

df.head()

,cases,deaths
date,,
2022-05-01,27.0,2.0
2022-05-02,27.0,2.0
2022-05-03,27.0,2.0
2022-05-04,27.0,2.0
2022-05-05,27.0,2.0


In [29]:
print(px.colors.qualitative.Dark24)

['#2E91E5', '#E15F99', '#1CA71C', '#FB0D0D', '#DA16FF', '#222A2A', '#B68100', '#750D86', '#EB663B', '#511CFB', '#00A08B', '#FB00D1', '#FC0080', '#B2828D', '#6C7C32', '#778AAE', '#862A16', '#A777F1', '#620042', '#1616A7', '#DA60CA', '#6C4516', '#0D2A63', '#AF0038']


In [33]:
fig = px.line(
    df,
    color_discrete_sequence=["#AB63FA", "#222A2A"],
    title="Mpox (monkeypox)"
)

fig.update_layout(legend_title="Who 🗿?")

fig.show()

In [4]:
df.index.max() - df.index.min(), len(df)

(Timedelta('591 days 00:00:00'), 592)

In [5]:
df_ca = df["cases"].to_numpy()
df_de = df["deaths"].to_numpy()

In [8]:
df_ca.shape, df_de.shape

((592,), (592,))

In [43]:
import plotly.graph_objects as go

def SEIR(y, t, beta, alpha, gamma, mu, N):
    S, E, I, R, D = y
    dS = - beta * S * I / N
    dE = beta * S * I / N - alpha * E
    dI = alpha * E - gamma * I - mu * I
    dR = gamma * I
    dD = mu * dI
    return [dS, dE, dI, dR, dD]

def fit(x, plot=False):
    I0 = 1          # initial number of infected individuals
    D0 = 0          # initial number of deceased individuals

    N = x[0]        # population
    beta = x[1]     # transmission rate (per day)
    alpha = x[2]    # incubation rate (per day)
    gamma = x[3]    # recovery rate (per day)
    mu = x[4]       # death rate (per day)
    E0 = x[5]       # initial number of exposed individuals
    R0 = x[6]       # initial number of recovered individuals

    S0 = N - E0 - I0 - R0 - D0
    t_max = 592      # maximum time to simulate (days)
    dt = 1           # time step size (days)

    # Initialize the simulation
    t = np.linspace(0, t_max, int(t_max / dt))
    y0 = [S0, E0, I0, R0, D0]

    # Integrate the ODEs using odeint
    sol = odeint(SEIR, y0, t, args=(beta, alpha, gamma, mu, N), atol=1e-8, rtol=1e-8)

    # Extract the solution values
    S, E, I, R, D = sol[:, :].T
    delta_I = mean_squared_error(df_ca, I, squared=False)
    delta_D = mean_squared_error(df_de, D, squared=False)

    if plot:
        fig = go.Figure()
        # Add traces
        fig.add_trace(go.Scatter(x=t, y=I,
                    mode='markers',
                    name='Infected pred',
                    marker_size=2,
                    marker_color='#AB63FA',
                    ))
        fig.add_trace(go.Scatter(x=t, y=D,
                    mode='markers',
                    name='Deceased pred',
                    marker_size=2,
                    marker_color='#222A2A',
                    ))
        fig.add_trace(go.Scatter(x=t, y=df_ca,
                    mode='lines',
                    name='Infected true',
                    marker_size=1,
                    marker_color='#AB63FA',
                    opacity=0.3,))
        fig.add_trace(go.Scatter(x=t, y=df_de,
                    mode='lines',
                    name='Deceased true',
                    marker_size=1,
                    marker_color='#222A2A',
                    opacity=0.3))
        
        fig.update_layout(
            title="Simulation of Monkey Pox Spread SEIRD",
            xaxis_title="Days",
            yaxis_title="People",
            legend_title="Who 🗿?",
        )
        fig.show()
    
    return np.sqrt(delta_I**2 + delta_D**2)

left = 6_500        # 2000 
right = 8_000_000_000   # 2023

x0 = np.array([(left+right)//2, 0.1, 0.1, 0.001, 0.1, 7000, 0])
#              N                beta alph gamm   mu   E0    R0
res = minimize(
    fit, x0, 
    method="nelder-mead",
    options={"xatol": 1e-8, "disp": True},
    bounds=[(left, right), (0, 1), (0, 1), (0, 1), (0, 1), (0, right), (0, right)]
)

print(f"N {res.x[0]:.1e}\tbeta {res.x[1]:.1e}\talpha {res.x[2]:.1e}\tgamma {res.x[3]:.1e}\tmu {res.x[4]:.1e}\tE0 {res.x[5]:.1e}\tR0 {res.x[6]:.1e}")

fit(res.x, plot=True)

N 9.1e+04	beta 2.9e-02	alpha 8.7e-01	gamma 5.8e-06	mu 1.0e-04	E0 3.1e+03	R0 0.0e+00


<ipython-input-43-6a4575b9b479>:83: RuntimeWarning:

Maximum number of function evaluations has been exceeded.



3832.745715507219